# Convergence Validation

G. Raush, 2022\
Department of Fluid Mechanics\
ESEIAAT-Terrassa\
UPC



Performs several verification calculations given a file of grid spacings    and some observed quantity corresponding to each grid spacing.
 
Computes:
   - Order of convergence
   - Richardson extrapolation to zero grid spacing
   - grid convergence indices (GCI)
 
 --------------------------------------------------------------------------
 
**Reference**:
- [Celik, 2008] Celik, I. B., Ghia, U., Roache, P. J., Freitas, C. J., Coleman, H., & Raad, P. E. (2008). Procedure for estimation and reporting of uncertainty due to discretization in CFD applications. Journal of Fluids Engineering, Transactions of the ASME, 130(7), 0780011–0780014. https://doi.org/10.1115/1.2960953
- Celik, I., Karatekin, O., 1997. Numerical Experiments on Application of Richardson Extrapolation With Nonuniform Grids. Journal of Fluids Engineering 119, 584–590. https://doi.org/10.1115/1.2819284
- [Roache, 1998] Roache, P. J. (1998). Verification of codes and calculations. AIAA Journal, 36(5), 696–702. https://doi.org/10.2514/2.457
- https://pypi.org/project/convergence/


In [8]:
import math

### Estimation of Discretization Error

Representative cell could be defined over a grid size $h$, for three-dimensional numerics

$$
h = \left[ \frac{1}{N} \sum_{i=1}^{N} \left(\Delta V_{i}\right)\right]^{1/3}
$$

For two dimensions,

$$
h = \left[ \frac{1}{N} \sum_{i=1}^{N} \left(\Delta A_{i}\right)\right]^{1/2}
$$


### Order of Convergence

\begin{align}
\epsilon_{21} &= \phi_2 - \phi_1\\
\epsilon_{32} &= \phi_3 - \phi_2\\
\end{align}

<!-- \begin{equation}
r = \frac{\epsilon_{32}}{\epsilon_{21}}
\end{equation}
-->

\begin{align}
e_r &= \frac{\epsilon_{32}}{\epsilon_{21}}\\
s &= \frac{e_r}{|e_r|}\\
\end{align}

\begin{align}
r_{21} &= \frac{\phi_{2}}{\phi_{1}}\\
r_{32} &= \frac{\phi_{3}}{\phi_{2}}\\
\end{align}

$$ p_0 = \frac{1}{\ln r_{21}} \left| \ln \left(\left| e_r \right| \right) \right|$$

$$ q = \ln \left( \frac{r_{21}^{p_0}- s}{r_{32}^{p_0} - s } \right) $$

$$ p_1 = \frac{1}{\ln r_{21}} \left| \ln \left(\left| e_r \right| + q \right) \right| \tag{1}$$

Linear interpolation, $\omega = 0.5$

$$ p_{1_{new}} = \left(1 - \omega\right) p_0 + \omega p_1 $$


In [9]:
def order_of_convergence (value_1, value_2, value_3, ratio_21, ratio_32, 
                          omega=0.5, tol=1.E-4): #, start_p=1.):
                              
    """ Calculate the order of convergence values generated with three
    grids of reducing resolution (ie grid_1 is finest). The values of the grids
    are needed along with the ratios between them.
    
    An iterative method with under-relaxation is used to calculate the order
    of convergence as the refinement ratio is not necessarily constant.
    
    This has been modified to the method of Celik (2008).
    """
    
    # Set a maximum residual and number of iterations
    max_res = 1.E6

    # calculate the epsilons.
    epsilon21 = float(value_2 - value_1)
    epsilon32 = float(value_3 - value_2)

    # Calculate the fraction
    epfrac = epsilon32 / epsilon21
    
    # Get the signed unit, s
    s = epfrac / abs(epfrac)
    
    # Initial guess at order of convergence, p
    p1 = (1. / math.log(ratio_21)) * abs(math.log(abs(epfrac))) # start_p
    
    # Initialise the residual and number of iterations
    residual = 1.
    iterations = 0
    
    while abs(residual) > tol:
    
        # Break if it's all gone bad
        if float(iterations) > max_res or residual > max_res:
            ValueError('Residual out of range or too many iterations')
        
        # Get the last value
        p0 = p1
        
        # Calculate q
        q = math.log((ratio_21 ** p0 - s) / (ratio_32 ** p0 - s))
        
        # Calculate the p iteration
        pnew = (1. / math.log(ratio_21)) * abs(math.log(abs(epfrac)) + q)
        
        # Calculate the relaxation step.
        p1 = (1. - omega) * p0 + omega * pnew
        
        residual = p1 - p0
        
        iterations += 1
        
    return p1

### Richardson's Extrapolation

$$\phi_{ext} = \frac{r_{21}^p \phi_1 - \phi_2}{r_{21}^p - 1 }\tag{2}$$

In [10]:
def richardson_extrapolate(value_1, value_2, ratio_21, p):
    
    """ Estimate the zero grid spacing value using richardsons extrapolation and
    two grids of reducing resolution (ie grid_1 is finest). The refinement ratio
    is needed. The order of convergence, p, is also required.
    """
    
    f_exact = ( ratio_21**p * value_1 - value_2 ) / ( ratio_21**p - 1.0 )
    
    return f_exact

\begin{align}
    er_{21}     &= \frac{\phi_1 - \phi_2}{\phi_1} \tag{3a}\\
    er_{21,ext} &= \frac{\phi_{ext} - \phi_1}{\phi_{ext}} \tag{3b}\\
\end{align}

In [11]:
def error_estimates(value_1, value_2, f_exact):
    
    """ This routine returns the relative error and extrapolated 
    relative error. The values of the grids and needed along 
    with the extrapolated value.
    """

    # Get the approximate relative error
    e21a =  abs( (value_1 - value_2) / value_1 )
    
    # Get the extrapolated relative error
    e21ext = abs( ( f_exact - value_1 ) / f_exact )
    
    return e21a, e21ext

#### Grid convergence Ratio, $GCI$

\begin{align}
GCI_{fine} &= 1.25 \frac{er_{21}}{r_{21}^p - 1}\tag{4a}\\
GCI_{coarse} &= 1.25 r_{21}^p \frac{er_{21}}{r_{21}^p - 1}\tag{4b}\\
\end{align}

In [12]:
def gci(ratio_21, e21_approx, p):
    
    """ Calculate the fine and coarse grid convergence index for two grids of 
    reducing resolution (ie grid_1 is finest). The refinement ration between the 
    grids is required along with the approximate relative error (e21_approx) and 
    the order of convergence, p.
    """
    
    # Using a fixed safety factor as per Celik (2008)
    safety_factor = 1.25
    
    # Calculate the gci
    gci_fine = safety_factor * e21_approx / (ratio_21**p - 1.0)

    gci_coarse = ratio_21**p * gci_fine
    
    return gci_fine, gci_coarse

#### Asymptotic Ratio

<!-- \begin{align}
GCI_{fine} &= 1.25 \frac{er_{21}}{r_{21}^p - 1}\\
GCI_{coarse} &= 1.25 r_{21}^p \frac{er_{21}}{r_{21}^p - 1}\\
\end{align}
-->
$$AR = r_{21}^p \frac{GCI_{21}}{GCI_{32}}$$


In [13]:
def asymptotic_ratio(gci_fine_21, gci_fine_32, ratio_21, p):
    
    """ Calculate the ratio in succesive Eps as defined at the bottom of page
    129 in Roache. If the ration is close to one then the asymptotic range has
    been reached.
    """
    
    ratio = ratio_21**p * ( gci_fine_21 / gci_fine_32)
    
    return ratio

### Test Data

### Proc

Examples from (_Celik 2008_)

**Monotonic convergence**

In [14]:
phi1 = 6.063
phi2 = 5.972
phi3 = 5.863
r21 = 1.5
r32 = 1.333

In [15]:
p = order_of_convergence(phi1,phi2,phi3,r21,r32)
p

1.537048621203307

In [16]:
phi21extrapolation = richardson_extrapolate(phi1,phi2,r21,p)
phi21extrapolation

6.168211718640093

In [17]:
err21aprox, err21extrap = error_estimates(phi1,phi2,phi21extrapolation)
print(list(map(lambda x : x*100,[err21aprox, err21extrap])))

[1.5009071416790254, 1.705708614413271]


In [18]:
gci21fine,gci21coarse = gci(r21,err21aprox,p)
print(list(map(lambda x : x*100,[gci21fine,gci21coarse])))

[2.169134888670891, 4.045268815769673]


**Case with** $p < 1$

In [19]:
def convergence(value1,value2,value3,ratio1,ratio2):
    p = order_of_convergence(value1,value2,value3,ratio1,ratio2)
    print('p = %1.2f' % p)
    phi21extrapolation = richardson_extrapolate(value1,value2,ratio1,p)
    print('phi21ext = %1.4f' % phi21extrapolation)
    err21aprox, err21extrap = error_estimates(value1,value2,phi21extrapolation)
    print('err21aprox = %1.1f%%, err21extrap = %1.1f%%' % 
      tuple(map(lambda x : x*100,[err21aprox, err21extrap])))
    gci21fine,gci21coarse = gci(ratio1,err21aprox,p)
    print('GCI21fine = %1.1f%%, GCI21coarse = %1.1f%%' % 
      tuple(map(lambda x : x*100,[gci21fine,gci21coarse])))

In [20]:
r21 = 2.0
r32 = 2.143
phi1 = 10.788
phi2 = 10.725
phi3 = 10.605

In [21]:
convergence(phi1,phi2,phi3,r21,r32)

p = 0.75
phi21ext = 10.8801
err21aprox = 0.6%, err21extrap = 0.8%
GCI21fine = 1.1%, GCI21coarse = 1.8%


**monotonic convergence, [Celik, 2008]**

In [22]:
r21 = 1.50
r32 = 1.333
phi1 = 6.063
phi2 = 5.972
phi3 = 5.863

p = order_of_convergence(phi1,phi2,phi3,r21,r32)
print('p = %1.2f' % p)
phi21extrapolation = richardson_extrapolate(phi1,phi2,r21,p)
print('phi21ext = %1.4f' % phi21extrapolation)
err21aprox, err21extrap = error_estimates(phi1,phi2,phi21extrapolation)
print('err21aprox = %1.1f%%, err21extrap = %1.1f%%' % 
      tuple(map(lambda x : x*100,[err21aprox, err21extrap])))
gci21fine,gci21coarse = gci(r21,err21aprox,p)
print('GCI21fine = %1.1f%%, GCI21coarse = %1.1f%%' % 
      tuple(map(lambda x : x*100,[gci21fine,gci21coarse])))

print('.....=====.....')
# Procedimeinto
convergence(phi1,phi2,phi3,r21,r32)

p = 1.54
phi21ext = 6.1682
err21aprox = 1.5%, err21extrap = 1.7%
GCI21fine = 2.2%, GCI21coarse = 4.0%
.....=====.....
p = 1.54
phi21ext = 6.1682
err21aprox = 1.5%, err21extrap = 1.7%
GCI21fine = 2.2%, GCI21coarse = 4.0%


### Comparatives

![Celik2008a](./Celik2008a.png)

_Celik,2008_

### Error reports

![Celik2008a](./Celik2008b.png)

_Celik, 2008_: "_Eq. 7_" in Figure corresponds to the above Expression 4a $GCI_{fine}$